In [ ]:
import pandas as pd
import numpy as np
import logging
import gc
#logging.disable()
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)
dirx1 = 'S:\C08\WAC\FO\CASSANDRA\DATENSAETZE\ESO_2021PreOP'
dir1 = dirx1 + "\pred_modelAll.xlsx"
df = pd.read_excel(open(dir1, 'rb'), sheet_name='sheet1', skiprows=0)
df = df[df['score'].notna()]
del dir1
# unterteile daten in 5 Risikogruppen
for idx, row in df.iloc[:].iterrows():
    score = df.loc[idx,'score']
    if score >= 1 :
        df.loc[idx,'very_low_risk'] = 1
    elif score == 0:
        df.loc[idx,'low_risk'] = 1
    elif score <= -1 and score >= -2:
        df.loc[idx,'medium_risk'] = 1
    elif score <=-3 and score >= -4:
        df.loc[idx,'high_risk'] = 1
    elif score <=-5:
        df.loc[idx,'very_high_risk'] = 1
    else:
        continue
del row, idx, score
#%% create dictionary with expected 90 day mortalities
dict1 = {'very_low_risk': 1.9, 'low_risk': 3, 'medium_risk': 5.8, 'high_risk': 8.9, 'very_high_risk': 18.2}
expected_mortalities = list(dict1.values())
risk_groups = list(dict1.keys())
logging.debug(len(list(dict1.keys())))
import math
from scipy.stats import chi2
daten = []
for i in range(len(risk_groups)):
    print("Risk group:",risk_groups[i])
    observed_data = []
    expected_data = []
    daten2 = []
    pearson = []
    df2 = df[[risk_groups[i],'90d_mortality']].copy()
    df2 = df2[df2[risk_groups[i]] == 1]

    observed_mortality_rate = round(df2['90d_mortality'].value_counts()[1]/len(df2)*100, 1)
    observed_mortality = df2['90d_mortality'].value_counts()[1]
    observed_survival = len(df2)-df2['90d_mortality'].value_counts()[1]
    #logging.debug(observed_mortality_rate)
    observed_data.extend([observed_mortality,observed_survival])
    print("observed mortality rate:", observed_mortality_rate,"%")
    print("observed mortality:", observed_mortality)
    print("observed survival:", observed_survival)
    a = math.floor(len(df2)*expected_mortalities[i]/100) # Anzahl der erwarteten 90 Tage Todesfälle
    #logging.debug(a)
    expected_mortality = a
    expected_survival = len(df2)-a
    print("Expected mortality rate:", expected_mortalities[i],"%")
    print("Expected mortality:", expected_mortality)
    print("Expected survival:", expected_survival)
    expected_data.extend([expected_mortality,expected_survival])
    #logging.debug(observed_data)
    #logging.debug(expected_data)
    chi_square_test_statistic, p_value = stats.chisquare(
    observed_data, expected_data)
    print('p_value : ' ,round(p_value,4))
    daten2.extend([risk_groups[i],observed_mortality_rate,expected_mortalities[i]])
    daten.append(daten2)
    ## bar chart
    pearson2=[]
    pearson2.extend([observed_mortality,expected_mortality])
    pearson.append(pearson2)
    pearson2=[]
    pearson2.extend([observed_survival,expected_survival])
    pearson.append(pearson2)
    #pearson.append(observed_data)
    exp =  pd.DataFrame(pearson, columns =['observed','expected']) 
    display(exp)
    tstat = np.sum(((df2-exp)**2/exp).values) # calculate chi-squared test statistic
    print(tstat)
    dof = (len(exp.columns)-1)*(len(exp.index)-1) # determine degrees of freedom 
    print(dof)
    # Apply Yates' correction by subtracting 0.5 from the absolute difference between observed and expected counts: 
    tstat_yates= np.sum((((np.abs(dof-exp)-0.01)**2)  / (exp)).values)
    print("Chi-squared test statistic with Yates correction: " + str(tstat_yates))

    pval= 1-   chi2.cdf(tstat_yates, dof)
    print("P-value with Yates correction: " + str(pval))
    print("\n")
    break
del i, a

df_result = pd.DataFrame(daten, columns =['Risk_group', 'observed_mortality_rate','expected_mortality_rate']) 
display(df_result )
df_result.plot(kind='bar' , x='Risk_group');
import math
from scipy.stats import chi2
daten = []
for i in range(len(risk_groups)):
    print("Risk group:",risk_groups[i])
    observed_data = []
    expected_data = []
    df2 = df[[risk_groups[i],'90d_mortality']].copy()
    df2 = df2[df2[risk_groups[i]] == 1]
    df3 = df[[risk_groups[i],'90d_mortality']].copy()
    df3 = df3[df3[risk_groups[i]] != 1]

    display(df3)
    break
